In [ ]:
import os
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import imageio
import tensorflow as tf
from tqdm import tqdm

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

MODELS_DIR = "/home/mel/datasets/outmark/"

In [ ]:
graph = tf.Graph()
session = tf.Session(graph=graph)

models = ["abdomen", "lits"]

with tf.device('/cpu:0'):
    with graph.as_default():
        for model in models:
            tf.train.import_meta_graph(MODELS_DIR + model + ".graphdef", import_scope=model)

        session.run(tf.global_variables_initializer())

In [ ]:
def extract_features(image):
    all_results = []
    for model in models:
        inputs = graph.get_tensor_by_name(model + "/X:0")
        is_training = graph.get_tensor_by_name(model + "/is_training_1:0")
        keep_prob = graph.get_tensor_by_name(model + "/keep_prob:0")
        features = graph.get_tensor_by_name(model + "/features:0")

        results = []
        for z in tqdm(range(image.shape[0])):
            X = np.expand_dims(np.expand_dims(image[z, :, :], axis=0), axis=1)
            f, = session.run([features], feed_dict={inputs: X, is_training: False, keep_prob: 0.5})
            
            results.append(f[0, 0, :, :])

        results = np.stack(results, axis=0)
        all_results.append(results)

    return np.concatenate(all_results, axis=-1)

assert extract_features(np.random.normal(size=(5,32,32))).shape[0:3] == (5, 32, 32)

In [ ]:
DATA_DIR = "/large/datasets/NF/"

example = scipy.io.loadmat(DATA_DIR + "NFdb_1013_5.mat")
example = example["Inh"][0][0]

In [ ]:
fig=plt.figure(figsize=(12, 12))

fig.add_subplot(1, 4, 1)
plt.imshow(example[1][200], cmap='gray')

fig.add_subplot(1, 4, 2)
plt.imshow(example[2][200], cmap='gray')

fig.add_subplot(1, 4, 3)
plt.imshow(example[3][200], cmap='gray')

fig.add_subplot(1, 4, 4)
plt.imshow(example[4][200], cmap='gray')


In [ ]:
X = np.swapaxes(example[2], 0, 2).copy()
Y = np.swapaxes(example[1], 0, 2).copy()

assert X.shape == Y.shape

pad_x_1 = (512 - X.shape[1])//2
pad_x_2 = 512 - X.shape[1] - pad_x_1
pad_y_1 = (512 - X.shape[2])//2
pad_y_2 = 512 - X.shape[2] - pad_y_1

X = np.pad(X, pad_width=[(0, 0), (pad_x_1, pad_x_2), (pad_y_1, pad_y_2)], mode="edge")
Y = np.pad(Y, pad_width=[(0, 0), (pad_x_1, pad_x_2), (pad_y_1, pad_y_2)], mode="edge")

Y[X == Y] = 0
X = (X - 1024.0) / 1024.0
Y = Y.astype(np.int32)

fig=plt.figure(figsize=(12, 12))

fig.add_subplot(2, 2, 1)
plt.imshow(X[100])

fig.add_subplot(2, 2, 2)
plt.hist(X.reshape(-1), bins = 20)

fig.add_subplot(2, 2, 3)
plt.imshow(Y[100])

fig.add_subplot(2, 2, 4)
plt.hist(Y.reshape(-1), bins = 20)

fig.show()

In [ ]:
F = extract_features(X[100:110])

In [ ]:
fig=plt.figure(figsize=(12, 12))
fig.add_subplot(2, 1, 1)
plt.hist(F[:, :, :, 0].reshape(-1))
fig.add_subplot(2, 1, 2)
plt.hist(X.reshape(-1));

In [ ]:
fig=plt.figure(figsize=(12, 12))

fig.add_subplot(2, 2, 1)
plt.imshow(X[100], cmap="gray")

fig.add_subplot(2, 2, 2)
plt.imshow(F[0, :, :, 90], cmap="gray")

fig.add_subplot(2, 2, 3)
plt.imshow(F[0, :, :, 10], cmap="gray")

fig.add_subplot(2, 2, 4)
plt.imshow(F[0, :, :, 110], cmap="gray")

fig.show()

In [ ]:
F_flat = F.reshape(-1, F.shape[-1])
Y_flat = Y[100:110].reshape(-1, 1)

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(F_flat, Y_flat, test_size=0.3)

clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X_train, y_train)

print(np.mean(clf.predict(X_test) == y_test))